In [53]:
import pandas as pd
import numpy as np
import seaborn as sns

**reason to why we use pipeline**
agar pipleline use nhi krte to humko baar baar code likhna padta coltransformer ke liye 
or production ke time bhi hume bht code likhna padta 

In [54]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [55]:
df = pd.read_csv(r"C:\Users\khushi pawar\Downloads\train.csv")

In [56]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [57]:
df.shape

(891, 12)

In [58]:
df=df.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [61]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [62]:
y_train.shape

(712,)

In [63]:
y_test.shape

(179,)

In [64]:
X_test.shape

(179, 7)

In [65]:
tnsi=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),#by_deafault mean lega
    ('impute_embarded',SimpleImputer(strategy='most_frequent'),[6])#isme humne jo sabse jyada baar aaiga usko lege strategy me
],remainder='passthrough')

**from sklearn.compose import ColumnTransformer,make_column_transformer

tnsi=make_column_transformer([


    (SimpleImputer(),[2]),#by_deafault mean lega

    (SimpleImputer(strategy='most_frequent'),[6])#isme humne jo sabse jyada baar aaiga usko lege strategy me
    
],remainder='passthrough')**

In [66]:
#col ke naam ke bjai hum indexing use kr rhe hai kyuki isse code fatega nhi mtlb error aa jata hai

In [86]:
X_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [121]:
tnohe=ColumnTransformer([
    ('encode_sex',OneHotEncoder(sparse_output=False,dtype=int,drop='first',handle_unknown='ignore'),[1,6])#handle_unknown => agar new col add hua to,agar humne handle_unkwon nhi use kiya hota to new cat agar add hoti to error de deta ,isko use krne se error nhi aaiga usko wo ignore kr dega
     ],remainder='drop')

In [122]:
tnsc=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)) 
],remainder='passthrough')

In [123]:
trf4 = SelectKBest(score_func=chi2,k=8) #feature selection
# train the model
trf5 = DecisionTreeClassifier()

In [124]:
from sklearn.pipeline import Pipeline,make_pipeline

**pipeline me do cheeje dalege (name of transformer,object of transformer)**

In [125]:
pipe = Pipeline([
    ('tnsi',tnsi),
    ('tnohe',tnohe),
    ('tnsc',tnsc),
    ('trf4',trf4),
    ('trf5',trf5)
])

**alternate way**
pipe = make_pipeline( tnsi , tnohe , tnsc , trf4 , trf5 )

In [126]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('tnsi',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarded',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tnohe',
                 ColumnTransformer(transformers=[('encode_sex',
                                                  OneHotEncoder(drop='first',
                                                                dtype=<class 'int'>,
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('tnsc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001E1A2960540>)),
                ('trf5', DecisionTreeClassifier())])

In [127]:
pipe.named_steps

{'tnsi': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarded',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tnohe': ColumnTransformer(transformers=[('encode_sex',
                                  OneHotEncoder(drop='first',
                                                dtype=<class 'int'>,
                                                handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'tnsc': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x000001E1A2960540>),
 'trf5': DecisionTreeClassifier()}

In [128]:
pipe.named_steps['tnsi'].transformers_[0][1].statistics_

array([29.49884615])

In [135]:
pipe.named_steps['tnohe'].transformers_[0][1]

OneHotEncoder(drop='first', dtype=<class 'int'>, handle_unknown='ignore',
              sparse_output=False)